In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
from urllib.parse import quote
from folium import *
from folium.plugins import MarkerCluster
import folium
import json

### 1. 데이터를 가져오기

In [ ]:
# url = pd.read_csv("c:/프로젝트/Hotel Review//link_1.csv")
# url.head()

In [ ]:
# site = url['link']

# details = []

# for link in links :
      

In [ ]:
path = "c:/python/chromedriver.exe"
driver = webdriver.Chrome()
url = 'https://hotels.naver.com/item/photo?hotelFileName=hotel%3AParnas_Hotel_Jeju&adultCnt=2&includeTax=false'
driver.get(url)
time.sleep(1)

driver.find_element(By.XPATH, r'//*[@id="__next"]/div/div/div[2]/div/a[5]').click()

In [ ]:
# url = f'https://hotels.naver.com/item/photo?hotelFileName=hotel%3AParnas_Hotel_Jeju&adultCnt=2&includeTax=false'
# req = requests.get(url)
# html = req.text

In [ ]:
# path = "c:/python/chromedriver.exe"
# driver = webdriver.Chrome()
# url = 'https://hotels.naver.com/item/photo?hotelFileName=hotel%3AParnas_Hotel_Jeju&adultCnt=2&includeTax=false'
# driver.get(url)

### 2. 원하는 데이터 찾기

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
lis = soup.find_all('li')
len(lis)

In [ ]:
# 호텔명
name = soup.select_one('.common_name__TacRt').get_text()
print(name)
# name = soup.select_one('.common_name__TacRt > strong > div' )
# len(lis)

In [ ]:
lis1 = soup.select_one(".common_ReviewList__4cL43").find_all('li')
# lis = soup.find_all('li')
len(lis1)

In [ ]:
lis = soup.select('.common_ReviewList__4cL43 > li' )
#lis = soup.find_all('li', attrs={"class":"common_item__BzEBu"}) 
len(lis)

In [ ]:
li = lis1[0]

In [ ]:
# 리뷰제목
#scroll()
#title = soup.select_one(".common_title_txt__Kn91_").get_text()
title = li.find(attrs={'class':'common_title_txt__Kn91_'}).get_text()
# for title in titles :
#     print(title.get_text())
print(title)

In [ ]:
# 평점
score = float(li.find('b', attrs={'class':'common_star_txt__3f1Yt'}).get_text())
# for score in scores :
#     print(score.get_text()) 
#score = soup.select_one("b.common_star_txt__3f1Yt").get_text() 
print(score)

In [ ]:
# 긍정리뷰
p_review=li.find('p',attrs={"class":"common_booking_txt___dAYT common_as_positive__8PlLV"}).get_text()
#review_p = soup.select_one("p.common_booking_txt___dAYT common_as_positive__8PlLV").string
# for p_review in p_reviews : 
#     print(p_review.get_text())
print(p_review)

In [ ]:
# 부정리뷰
n_review = li.find('p',attrs={"class":"common_booking_txt___dAYT common_as_negative__LjbBo"}).get_text() 
# for n_review in n_reviews :
#     print(n_review.get_text()) 
print(n_review)

In [ ]:
# place = li.select_one('.cate').get_text().split('/')[0].strip()
# place

### 3. 반복문으로 데이터 전부 가져오기

In [ ]:
# 스크롤 수 지정하기
def scroll () :
    #last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    before_location = driver.execute_script("return window.pageYOffset")
    while True:
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0,{})".format(scroll_location + 40))
        # 끝까지 스크롤 내리기
        time.sleep(60)
        # 스크롤 내린 후 스크롤 높이 다시 가져옴
        #new_height = driver.execute_script("return document.body.scrollHeight")
        after_location = driver.execute_script("return window.pageYOffset")
        #if new_height == last_height:
        if before_location == after_location:
            break
        else :
            before_location = driver.execute_script("return window.pageYOffset")
        #last_height = new_height

In [ ]:
# 반복문으로 가져오기
scroll()
line = []

for li in lis1:
    #name = li.select_one('.common_name__TacRt' ).get_text() 
    title = li.find(attrs={'class':'common_title_txt__Kn91_'}).get_text()
    score = li.find('b', attrs={'class':'common_star_txt__3f1Yt'}).get_text()
    try : 
        p_review=li.find('p',attrs={"class":"common_booking_txt___dAYT common_as_positive__8PlLV"}).get_text()
    except:
        p_review = "없음"
    try : 
        n_review = li.find('p',attrs={"class":"common_booking_txt___dAYT common_as_negative__LjbBo"}).get_text()
    except:
        n_review = "없음"   
    line.append({'title':title , 'score': score , 'p_review':p_review , 'n_review':n_review })

driver.close()

### 4. DF 생성  및 저장

In [ ]:
df= pd.DataFrame(line)
df.head()

In [ ]:
df['name']=name
df.head()

In [ ]:
df.to_csv('c:/프로젝트/Hotel Review/review_2.csv',encoding="utf-8-sig",index=False)